This code is given in https://github.com/allan-tulane/CMPS4661-6663/blob/main/Image_Stitching.ipynb
Most of them are implemented using existing libraries

The below one is simplified to panorama over multiple images.
There are 5 steps listed in the program including
- Step 1: read two images and resize to the same size
- Step 2: detect the features and keypoints from SIFT
- Step 3: get the valid matched points out of all
- Step 4: Call cv2.findHomography to calculate the 3x3 H matrix
- Step 5: to get perspective of image using computed homography

## To Do
**Task 1**: This aims to reimplement Step 4 instead of using *cv2.findHomography*, which calcutlates the H matrix (3x3) using RANSAC given all matched pairs from image A and image B. You will implement two ways to calculate H.

- First, check the Concept *The direct linear transform (DLT)*. Please directly implement DLT (Page 50 in the slides) using all pairs to calculate H, which serves as the baseline.

- Second, check *Estimating homography using RANSAC* to recalculate H. When we count inliers, we can use Squared SUM Error (SSE) to determine each line. That is, given a pair (p, q), we can calculate the SSE of the converted Hp, and q. If their SSE is smaller than a threshold, that is an inlier. The threshold is manually set. You can try different values.

Note that you first use the given two images, then try to use the three images in folder **Sample**. When we have multiple images, we should stitch every two images sequentially.

**Task 2**: There are several parameters like lowe_ratio in def All_validmatches(AllMatches, lowe_ratio), and how many iterations in RANSAC for H. Please try different values to see the difference of output.

**Task 3**: The given code does a hard seem, which can not given a smooth result when two images have different illumination, exposures. Can you explore to achieve better results? Please implement the methods we discuss in class like **choose seaming** and **blending** like **alpha blending**, **multi-band blending as Laplacian Pyramid**. Please use the two pairs of images in folder of **Sample2**.




In [2]:
## to access the google drive with the google account
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import numpy as np
import imutils
import cv2
from matplotlib import pyplot as plt

def Detect_Feature_And_KeyPoints(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect and extract features from the image
    descriptors = cv2.xfeatures2d.SIFT_create()
    (Keypoints, features) = descriptors.detectAndCompute(image, None)

    Keypoints = np.float32([i.pt for i in Keypoints])
    return (Keypoints, features)

def get_Allpossible_Match(featuresA,featuresB):

    # compute the all matches using euclidean distance and opencv provide
    #DescriptorMatcher_create() function for that
    match_instance = cv2.DescriptorMatcher_create("BruteForce")
    All_Matches = match_instance.knnMatch(featuresA, featuresB, 2)

    return All_Matches

def All_validmatches(AllMatches, lowe_ratio):
    #to get all valid matches according to lowe concept..
    valid_matches = []

    for val in AllMatches:
        if len(val) == 2 and val[0].distance < val[1].distance * lowe_ratio:
            valid_matches.append((val[0].trainIdx, val[0].queryIdx))

    return valid_matches

def getwarp_perspective(imageA,imageB,Homography):
    ## given two images and H matrix to generate the panaroma
    val = imageA.shape[1] + imageB.shape[1]
    result_image = cv2.warpPerspective(imageA, Homography, (val , imageA.shape[0]))

    return result_image

def draw_Matches(imageA, imageB, KeypointsA, KeypointsB, matches, status):

    (hA,wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB

    # loop over the matches
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        if s == 1:
            ptA = (int(KeypointsA[queryIdx][0]), int(KeypointsA[queryIdx][1]))
            ptB = (int(KeypointsB[trainIdx][0]) + wA, int(KeypointsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)


def construct_A_matrix(valid_matches, points_A, points_B):
    A = []
    for match in valid_matches:
        trainIdx = match[0]
        queryIdx = match[1]
        x, y = points_A[queryIdx]
        x_prime, y_prime = points_B[trainIdx]
        row1 = [x, y, 1, 0, 0, 0, -x_prime*x, -x_prime*y, -x_prime]
        row2 = [0, 0, 0, x, y, 1, -y_prime*x, -y_prime*y, -y_prime]
        A.append(row1)
        A.append(row2)
    A = np.array(A)
    return A

def compute_homography(A):
    U, S, Vt = np.linalg.svd(A)

    h = Vt[-1, :]

    H = h.reshape(3, 3)

    return H

In [4]:
# Step 1: read images and resize images
filename = ['q11.jpg', 'q22.jpg']

images = []
img_path = '/content/drive/My Drive/Colab Notebooks/Assignment3/'
for i in range(len(filename)):
    ## read image and append
    images.append(cv2.imread(img_path+filename[i]))
    ## resize image to 2500x2500
    images[i] = cv2.resize(images[i], (2500, 2500))



#Step 2: detect the features and keypoints from SIFT
(imageB, imageA) = images
(KeypointsA, featuresA) = Detect_Feature_And_KeyPoints(imageA)
(KeypointsB, featuresB) = Detect_Feature_And_KeyPoints(imageB)

#Step 3: get the valid matched points
AllMatches = get_Allpossible_Match(featuresA, featuresB);
valid_matches = All_validmatches(AllMatches, lowe_ratio=0.75)

#Step 4: Call cv2.findHomography
## the goal is to calculate the 3x3 H matrix

print(len(valid_matches))
print(len(KeypointsA))
print(len(featuresA))


A=construct_A_matrix(valid_matches,KeypointsA,KeypointsB)
Hmatrix=compute_homography(A)
print(Hmatrix)


# construct the two sets of points
pointsA = np.float32([KeypointsA[i] for (_,i) in valid_matches])
pointsB = np.float32([KeypointsB[i] for (i,_) in valid_matches])

(Homograpgy, status) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC,4.0)
(Homograpgy2, status2) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC,40.0)
print(Homograpgy)
print(Homograpgy2)
## Step 5: to get perspective of image using computed homography
result_image = getwarp_perspective(imageA, imageB, Homograpgy)
result_image[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

draw_Matches(imageA, imageB, KeypointsA, KeypointsB, valid_matches, status)

## Save the images

cv2.imwrite(img_path+"image_output.jpg",result_image)

##test
status2 = [1] * len(valid_matches)
result_image = getwarp_perspective(imageA, imageB, Hmatrix)
result_image[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
output_image = draw_Matches(imageA, imageB, KeypointsA, KeypointsB, valid_matches, status2)
cv2.imwrite(img_path+"image_output2.jpg",result_image)




524
6555
6555
[[ 1.11628768e-04  4.63076417e-04 -7.95241319e-01]
 [ 1.11217268e-04  3.49905691e-04 -6.06292547e-01]
 [ 8.97363700e-08  2.66756109e-07 -4.78844035e-04]]
[[ 6.48804151e-01  1.43937419e-02  1.21561229e+03]
 [-1.82943537e-01  9.15052799e-01  1.10793552e+02]
 [-1.40677264e-04  4.51871572e-06  1.00000000e+00]]
[[ 6.55182329e-01  1.45746339e-02  1.20345241e+03]
 [-1.71865885e-01  8.96484736e-01  1.18474818e+02]
 [-1.32295525e-04 -4.92913270e-06  1.00000000e+00]]


True

**Sample test a+b**

In [5]:
# Step 1: read images and resize images
filenames = ['scene1_a.jpg', 'scene1_b.jpg']

images = []
img_path = '/content/drive/My Drive/Colab Notebooks/Assignment3/Sample/'

for filename in filenames:

   img = cv2.imread(img_path + filename)
   height, width = img.shape[:2]
   new_height = 2000
   aspect_ratio = width / height
   new_width = int(new_height * aspect_ratio)
   img_resized = cv2.resize(img, (new_width, new_height))
   images.append(img_resized)





#Step 2: detect the features and keypoints from SIFT
(imageB, imageA) = images
(KeypointsA, featuresA) = Detect_Feature_And_KeyPoints(imageA)
(KeypointsB, featuresB) = Detect_Feature_And_KeyPoints(imageB)

#Step 3: get the valid matched points
AllMatches = get_Allpossible_Match(featuresA, featuresB);
valid_matches = All_validmatches(AllMatches, lowe_ratio=0.75)

#Step 4: Call cv2.findHomography
## the goal is to calculate the 3x3 H matrix



# construct the two sets of points
pointsA = np.float32([KeypointsA[i] for (_,i) in valid_matches])
pointsB = np.float32([KeypointsB[i] for (i,_) in valid_matches])

(Homograpgy, status) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC,5.0)

## Step 5: to get perspective of image using computed homography
result_image = getwarp_perspective(imageA, imageB, Homograpgy)



result_image[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
result_image.resize()
cv2.imwrite(img_path+"ab.jpg",result_image)



True

**Sample test b+c**

In [5]:
# Step 1: read images and resize images
filenames = ['scene1_b.jpg', 'scene1_c.jpg']

images = []
img_path = '/content/drive/My Drive/Colab Notebooks/Assignment3/Sample/'

for filename in filenames:

   img = cv2.imread(img_path + filename)
   height, width = img.shape[:2]
   new_height = 2000
   aspect_ratio = width / height
   new_width = int(new_height * aspect_ratio)
   img_resized = cv2.resize(img, (new_width, new_height))
   images.append(img_resized)





#Step 2: detect the features and keypoints from SIFT
(imageB, imageA) = images
(KeypointsA, featuresA) = Detect_Feature_And_KeyPoints(imageA)
(KeypointsB, featuresB) = Detect_Feature_And_KeyPoints(imageB)

#Step 3: get the valid matched points
AllMatches = get_Allpossible_Match(featuresA, featuresB);
valid_matches = All_validmatches(AllMatches, lowe_ratio=0.75)

#Step 4: Call cv2.findHomography
## the goal is to calculate the 3x3 H matrix



# construct the two sets of points
pointsA = np.float32([KeypointsA[i] for (_,i) in valid_matches])
pointsB = np.float32([KeypointsB[i] for (i,_) in valid_matches])

(Homograpgy, status) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC,5.0)

## Step 5: to get perspective of image using computed homography
result_image = getwarp_perspective(imageA, imageB, Homograpgy)



result_image[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
result_image.resize()
cv2.imwrite(img_path+"bc.jpg",result_image)



True

**Task2**

In [15]:
filename = ['q11.jpg', 'q22.jpg']

images = []
img_path = '/content/drive/My Drive/Colab Notebooks/Assignment3/'
for i in range(len(filename)):
    ## read image and append
    images.append(cv2.imread(img_path+filename[i]))
    ## resize image to 2500x2500
    images[i] = cv2.resize(images[i], (2500, 2500))



#Step 2: detect the features and keypoints from SIFT
(imageB, imageA) = images
(KeypointsA, featuresA) = Detect_Feature_And_KeyPoints(imageA)
(KeypointsB, featuresB) = Detect_Feature_And_KeyPoints(imageB)

#Step 3: get the valid matched points
AllMatches = get_Allpossible_Match(featuresA, featuresB);
valid_matches = All_validmatches(AllMatches, lowe_ratio=3)

#Step 4: Call cv2.findHomography
## the goal is to calculate the 3x3 H matrix


# construct the two sets of points
pointsA = np.float32([KeypointsA[i] for (_,i) in valid_matches])
pointsB = np.float32([KeypointsB[i] for (i,_) in valid_matches])

(Homograpgy, status) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC,4.0)

## Step 5: to get perspective of image using computed homography
result_image = getwarp_perspective(imageA, imageB, Homograpgy)
result_image[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

draw_Matches(imageA, imageB, KeypointsA, KeypointsB, valid_matches, status)

## Save the images

cv2.imwrite(img_path+"image_outputtask2.jpg",result_image)




True

**Task3**

In [9]:
import numpy as np
import imutils
import cv2
from matplotlib import pyplot as plt

def Detect_Feature_And_KeyPoints(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect and extract features from the image
    descriptors = cv2.SIFT_create()
    (Keypoints, features) = descriptors.detectAndCompute(image, None)

    Keypoints = np.float32([i.pt for i in Keypoints])
    return (Keypoints, features)

def get_Allpossible_Match(featuresA,featuresB):

    # compute the all matches using euclidean distance and opencv provide
    #DescriptorMatcher_create() function for that
    match_instance = cv2.DescriptorMatcher_create("BruteForce")
    All_Matches = match_instance.knnMatch(featuresA, featuresB, 2)

    return All_Matches

def All_validmatches(AllMatches, lowe_ratio):
    #to get all valid matches according to lowe concept..
    valid_matches = []

    for val in AllMatches:
        if len(val) == 2 and val[0].distance < val[1].distance * lowe_ratio:
            valid_matches.append((val[0].trainIdx, val[0].queryIdx))

    return valid_matches

def getwarp_perspective(imageA,imageB,Homography):
    ## given two images and H matrix to generate the panaroma
    val = imageA.shape[1] + imageB.shape[1]
    result_image = cv2.warpPerspective(imageA, Homography, (val , imageA.shape[0]))

    return result_image

def draw_Matches(imageA, imageB, KeypointsA, KeypointsB, matches, status):

    (hA,wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB

    # loop over the matches
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        if s == 1:
            ptA = (int(KeypointsA[queryIdx][0]), int(KeypointsA[queryIdx][1]))
            ptB = (int(KeypointsB[trainIdx][0]) + wA, int(KeypointsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)


def construct_A_matrix(valid_matches, points_A, points_B):
    A = []
    for match in valid_matches:
        trainIdx = match[0]
        queryIdx = match[1]
        x, y = points_A[queryIdx]
        x_prime, y_prime = points_B[trainIdx]
        row1 = [x, y, 1, 0, 0, 0, -x_prime*x, -x_prime*y, -x_prime]
        row2 = [0, 0, 0, x, y, 1, -y_prime*x, -y_prime*y, -y_prime]
        A.append(row1)
        A.append(row2)
    A = np.array(A)
    return A

def compute_homography(A):
    U, S, Vt = np.linalg.svd(A)

    h = Vt[-1, :]

    H = h.reshape(3, 3)

    return H

filename = ['r11.jpg', 'r22.jpg']

images = []
img_path = '/content/drive/My Drive/Colab Notebooks/Assignment3/Sample2/'


for i in range(len(filename)):
    ## read image and append
    images.append(cv2.imread(img_path+filename[i]))
    ## resize image to 2500x2500
    images[i] = cv2.resize(images[i], (2500, 2500))



#Step 2: detect the features and keypoints from SIFT
(imageB, imageA) = images
(KeypointsA, featuresA) = Detect_Feature_And_KeyPoints(imageA)
(KeypointsB, featuresB) = Detect_Feature_And_KeyPoints(imageB)

#Step 3: get the valid matched points
AllMatches = get_Allpossible_Match(featuresA, featuresB);
valid_matches = All_validmatches(AllMatches, lowe_ratio=0.75)

#Step 4: Call cv2.findHomography
## the goal is to calculate the 3x3 H matrix


pointsA = np.float32([KeypointsA[i] for (_,i) in valid_matches])
pointsB = np.float32([KeypointsB[i] for (i,_) in valid_matches])

(Homograpgy, status) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC,4.0)


def alpha_blend(imageA, imageB, Homography, alpha=0.7):
    warped_imageB = cv2.warpPerspective(imageB, Homography, (imageA.shape[1], imageA.shape[0]))
    maskB = np.any(warped_imageB > 0, axis=-1).astype(np.float32)
    maskB = cv2.GaussianBlur(maskB, (21, 21), 0)
    maskA = 1.0 - maskB
    maskA = np.repeat(maskA[:, :, np.newaxis], 3, axis=2)
    maskB = np.repeat(maskB[:, :, np.newaxis], 3, axis=2)
    blended = (imageA * maskA * alpha + warped_imageB * maskB * (1 - alpha)).astype(np.uint8)
    return blended

blended_image = alpha_blend(imageB, imageA, Homograpgy, alpha=0.5)

cv2.imwrite(img_path+"task3.jpg", blended_image)

True